In [1]:
import glob

import sys
sys.path.append('/export/usuarios_ml4ds/sblanco/readTEDXML/pySpark/lib')
from utils import openTarFile
from utils import TranslateMachine

from pyspark.sql import Row


In [2]:
year = 2018
path ='///export/data_ml4ds/IntelComp/Datasets/ted/xmls/'
filePattern = path + '*' + str(year) + '*.tar.gz'

totalResult = []
tm = TranslateMachine ()

#Procesamos todos los formularios que empiecen por F
validForm = ['F']

for file in glob.glob( filePattern ):
    print (file)
    dataFile = openTarFile (file)
    print ('Filtrando formularios, en el original hay %s datos' % len(dataFile))
    dataFile = list(filter (lambda d: list (d['TED_EXPORT']['FORM_SECTION'].keys())[0][:1] in validForm, dataFile))
    print ('nos quedamos con %s datos' % len (dataFile))
    print ('convirtiendo datos')
    totalResult += [tm.translateKeys (data) for data in dataFile]






///export/data_ml4ds/IntelComp/Datasets/ted/xmls/2018-02.tar.gz
Filtrando formularios, en el original hay 45838 datos
nos quedamos con 36330 datos
convirtiendo datos
///export/data_ml4ds/IntelComp/Datasets/ted/xmls/2018-09.tar.gz
Filtrando formularios, en el original hay 44349 datos
nos quedamos con 37935 datos
convirtiendo datos
///export/data_ml4ds/IntelComp/Datasets/ted/xmls/2018-12.tar.gz
Filtrando formularios, en el original hay 49294 datos
nos quedamos con 41820 datos
convirtiendo datos
///export/data_ml4ds/IntelComp/Datasets/ted/xmls/2018-04.tar.gz
Filtrando formularios, en el original hay 45166 datos
nos quedamos con 36834 datos
convirtiendo datos
///export/data_ml4ds/IntelComp/Datasets/ted/xmls/2018-10.tar.gz
Filtrando formularios, en el original hay 55345 datos
nos quedamos con 47476 datos
convirtiendo datos
///export/data_ml4ds/IntelComp/Datasets/ted/xmls/2018-06.tar.gz
Filtrando formularios, en el original hay 49327 datos
nos quedamos con 41227 datos
convirtiendo datos
///e

In [ ]:
###################################PRUEBAS 

file ='///export/data_ml4ds/IntelComp/Datasets/ted/pruebas/test.tgz'
file = '///export/data_ml4ds/IntelComp/Datasets/ted/xmls/2022-03.tar.gz'
totalResult = []

print ('abriendo fichero')
dataFile = openTarFile (file)

tm = TranslateMachine ()

print ('Filtrando formularios, en el original hay %s datos' % len(dataFile))
dataFile = list(filter (lambda d: list (d['TED_EXPORT']['FORM_SECTION'].keys())[0][:1] in validForm, dataFile))
print ('nos quedamos con %s datos' % len (dataFile))

print ('convirtiendo datos')
totalResult += [tm.translateKeys (data) for data in dataFile]




In [ ]:
totalResult = []

In [3]:
from pyspark.sql import Row

df = spark.createDataFrame(totalResult)



In [ ]:
df.take(10)

In [ ]:
data2 = list (filter (lambda x:type (x) ==list , data))
data2

In [ ]:
#df = spark.createDataFrame(totalResult[12344:12346])
data = totalResult[12344:12346]
data[1]

In [5]:
savePath = '/export/ml4ds/temporal/teddata/' + str(year)


df.write.mode("overwrite").parquet (savePath)

22/06/08 23:02:41 WARN scheduler.TaskSetManager: Stage 2 contains a task of very large size (23299 KiB). The maximum recommended task size is 1000 KiB.


In [6]:
patata = df.count()

22/06/08 23:06:14 WARN scheduler.TaskSetManager: Stage 3 contains a task of very large size (23299 KiB). The maximum recommended task size is 1000 KiB.


In [4]:
df.filter('ORIGINAL_LG="ES"').count()

22/06/08 22:59:10 WARN scheduler.TaskSetManager: Stage 0 contains a task of very large size (23299 KiB). The maximum recommended task size is 1000 KiB.


22873

In [7]:
patata

479439